In [1]:
import chess.pgn
import re
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
def parse_eval(comment):
    """Extrae la evaluación del motor desde un comentario PGN"""
    if not comment:
        return None
    match = re.search(r'\[%eval (-?\d+\.?\d*|\#-?\d+)', comment)
    if match:
        eval_str = match.group(1)
        if eval_str.startswith('#'):
            mate_in = int(eval_str[1:])
            return 100 if mate_in > 0 else -100
        else:
            return float(eval_str)
    return None

In [3]:
def parse_clock(comment):
    """Extrae el tiempo restante del reloj desde un comentario PGN"""
    if not comment:
        return None
    match = re.search(r'\[%clk (\d+):(\d+):(\d+(\.\d+)?)\]', comment)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        seconds = float(match.group(3))
        return hours * 3600 + minutes * 60 + seconds
    return None

In [4]:
def elo_to_class(elo):
    """Convierte un valor ELO numérico en una categoría (clase) textual"""
    if elo < 1500:
        return "Principiante"
    elif elo < 2000:
        return "Intermedio"
    elif elo < 2500:
        return "Avanzado"
    elif elo < 2900:
        return "Maestro Fide"
    else:
        return "Gran Maestro"


In [5]:
def estimate_accuracy(blunders, mistakes, inaccuracies, total_moves):
    """Estima la precisión del jugador basada en los errores."""
    if total_moves == 0:
        return 0.0
    penalties = (blunders * 1.0 + mistakes * 0.6 + inaccuracies * 0.3)
    raw_accuracy = 1 - (penalties / total_moves)
    return round(max(0.0, raw_accuracy * 100), 2)

In [6]:
def process_player(game, color):
    # (Tu función process_player sin los prints de depuración)
    data = {}
    headers = game.headers
    prefix = "White" if color == "white" else "Black"

    def has_eval_internal(game):
        node = game
        while node.variations:
            node = node.variation(0)
            if "[%eval" in node.comment:
                return True
        return False

    if not has_eval_internal(game):
        return None

    elo_str = headers.get(f"{prefix}Elo", "0")
    if not elo_str.isdigit():
        return None
    data["elo"] = int(elo_str)
    data["elo_class"] = elo_to_class(int(elo_str))

    tc = headers.get("TimeControl", "")
    if "+" in tc:
        base, inc = tc.split("+")
        data["time_base"] = int(base) if base.isdigit() else 0
        data["time_increment"] = int(inc) if inc.isdigit() else 0
    else:
        data["time_base"] = int(tc) if tc.isdigit() else 0
        data["time_increment"] = 0

    evals = []
    move_times = []
    node = game
    move_number = 0
    brilliant_moves_ann = excellent_moves_ann = interesting_moves_ann = 0
    mistake_moves_ann = blunder_moves_ann = 0
    captures = checks = 0

    while node.variations:
        node = node.variation(0)
        san = node.san()
        is_player_move = (color == "white" and move_number % 2 == 0) or (color == "black" and move_number % 2 == 1)
        if is_player_move:
            eval_score = parse_eval(node.comment)
            clock_time = parse_clock(node.comment)
            if eval_score is not None:
                evals.append(eval_score)
            if clock_time is not None:
                move_times.append(clock_time)
            if "!!" in san: brilliant_moves_ann += 1
            elif "??" in san: blunder_moves_ann += 1
            elif "?!" in san: interesting_moves_ann += 1
            elif "?" in san: mistake_moves_ann += 1
            elif "!" in san: excellent_moves_ann += 1
            if "x" in san: captures += 1
            if "+" in san: checks += 1
        move_number += 1

    if not evals:
        return None

    eval_diffs = [evals[i+1] - evals[i] for i in range(len(evals)-1)]
    abs_diffs = [abs(d) for d in eval_diffs]

    data["eval_mean"] = np.mean(evals)
    data["eval_std"] = np.std(evals)
    data["eval_median"] = np.median(evals)
    data["eval_range"] = max(evals) - min(evals)
    data["blunders"] = sum(1 for d in abs_diffs if d > 2)
    data["mistakes"] = sum(1 for d in abs_diffs if 1 < d <= 2)
    data["inaccuracies"] = sum(1 for d in abs_diffs if 0.5 < d <= 1)
    data["blunder_rate"] = data["blunders"] / len(evals) if evals else 0
    data["mistake_rate"] = data["mistakes"] / len(evals) if evals else 0
    data["brilliant_moves_ann"] = brilliant_moves_ann
    data["excellent_moves_ann"] = excellent_moves_ann
    data["interesting_moves_ann"] = interesting_moves_ann
    data["mistake_moves_ann"] = mistake_moves_ann
    data["blunder_moves_ann"] = blunder_moves_ann
    n = len(evals)
    opening = evals[:max(1, n//3)]
    midgame = evals[max(1, n//3):max(2, 2*n//3)]
    endgame = evals[max(2, 2*n//3):]
    data["opening_eval_mean"] = np.mean(opening) if opening else 0
    data["midgame_eval_mean"] = np.mean(midgame) if midgame else 0
    data["endgame_eval_mean"] = np.mean(endgame) if endgame else 0
    data["variance_opening_mid_end"] = np.std([np.mean(x) for x in [opening, midgame, endgame] if len(x) > 0]) if sum(len(x) > 0 for x in [opening, midgame, endgame]) > 1 else 0
    data["captures"] = captures
    data["checks"] = checks
    if move_times:
        data["avg_time_per_move"] = np.mean(move_times)
        data["time_std"] = np.std(move_times)
    else:
        data["avg_time_per_move"] = 0
        data["time_std"] = 0
    recoveries = sum(1 for i in range(len(evals)-1) if evals[i] < -1.5 and evals[i+1] > evals[i] + 0.5)
    data["recovery_rate"] = recoveries / sum(1 for eval in evals[:-1] if eval < -1.5) if any(eval < -1.5 for eval in evals[:-1]) else 0
    data["total_moves"] = len(evals)
    data["estimated_accuracy"] = estimate_accuracy(
        data["blunders"],
        data["mistakes"],
        data["inaccuracies"],
        len(evals)
    )
    return data

con comentario 

In [ ]:
def parse_pgn_and_limit_after(file_path, limit_per_class=50000):
    all_player_data = []
    count_per_class = defaultdict(int)
    total_games_read = 0
    elo_classes = set(elo_to_class(elo_level * 500) for elo_level in range(3, 7))

    with open(file_path, encoding='utf-8') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            total_games_read += 1
            if total_games_read % 1000 == 0:
                print(f"Leídas {total_games_read} partidas...")
                print("Conteo actual por clase:")
                for cls, count in count_per_class.items():
                    print(f"  {cls}: {count}")

            white_features = process_player(game, "white")
            if white_features:
                elo_class = white_features.get("elo_class")
                if elo_class:
                    white_features["player_color"] = "white"
                    white_features["time_control"] = str(game.headers.get("TimeControl", ""))
                    white_features["opening_eco"] = str(game.headers.get("ECO", ""))
                    white_features["opening_name"] = str(game.headers.get("Opening", ""))
                    white_features["result"] = str(game.headers.get("Result", ""))
                    if count_per_class[elo_class] < limit_per_class:
                        all_player_data.append(white_features)
                        count_per_class[elo_class] += 1

            black_features = process_player(game, "black")
            if black_features:
                elo_class = black_features.get("elo_class")
                if elo_class:
                    black_features["player_color"] = "black"
                    black_features["time_control"] = str(game.headers.get("TimeControl", ""))
                    black_features["opening_eco"] = str(game.headers.get("ECO", ""))
                    black_features["opening_name"] = str(game.headers.get("Opening", ""))
                    black_features["result"] = str(game.headers.get("Result", ""))
                    if count_per_class[elo_class] < limit_per_class:
                        all_player_data.append(black_features)
                        count_per_class[elo_class] += 1

            all_limits_reached = all(count >= limit_per_class for count in count_per_class.values())
            if all_limits_reached:
                print("Se ha alcanzado el límite de ejemplos por categoría.")
                break

    df = pd.DataFrame(all_player_data)
    print("Número de ejemplos por categoría (antes del límite estricto):")
    print(df['elo_class'].value_counts())

    limited_data = []
    grouped = df.groupby('elo_class')
    for name, group in grouped:
        limited_data.extend(group.head(limit_per_class))

    df_limited = pd.DataFrame(limited_data)
    print("\nNúmero de ejemplos por categoría (después del límite estricto):")
    print(df_limited['elo_class'].value_counts())
    print(f"Total de ejemplos en el DataFrame final: {len(df_limited)}")

    return df_limited


In [7]:
def parse_pgn_and_limit_after(file_path, limit_per_class=50000):
    """
    Analiza un archivo PGN, procesa solo partidas con evaluaciones,
    y luego limita el número de ejemplos por cada categoría de ELO.
    """
    all_player_data = []
    total_games_read = 0

    with open(file_path, encoding='utf-8') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            total_games_read += 1
            if total_games_read % 1000 == 0:
                print(f"Leídas {total_games_read} partidas...")

            white_features = process_player(game, "white")
            if white_features:
                white_features["player_color"] = "white"
                white_features["time_control"] = str(game.headers.get("TimeControl", ""))
                white_features["opening_eco"] = str(game.headers.get("ECO", ""))
                white_features["opening_name"] = str(game.headers.get("Opening", ""))
                white_features["result"] = str(game.headers.get("Result", ""))
                all_player_data.append(white_features)

            black_features = process_player(game, "black")
            if black_features:
                black_features["player_color"] = "black"
                black_features["time_control"] = str(game.headers.get("TimeControl", ""))
                black_features["opening_eco"] = str(game.headers.get("ECO", ""))
                black_features["opening_name"] = str(game.headers.get("Opening", ""))
                black_features["result"] = str(game.headers.get("Result", ""))
                all_player_data.append(black_features)

    df = pd.DataFrame(all_player_data)
    print(f"Total de ejemplos iniciales (con evaluaciones): {len(df)}")

    # Aplicar límite por clase de ELO
    limited_data = []
    grouped = df.groupby('elo_class')
    for name, group in grouped:
        limited_data.extend(group.head(limit_per_class))

    df_limited = pd.DataFrame(limited_data)
    print("\nNúmero de ejemplos por categoría (después del límite):")
    print(df_limited['elo_class'].value_counts())
    print(f"Total de ejemplos en el DataFrame final: {len(df_limited)}")

    return df_limited

In [ ]:
# Ruta al archivo PGN
file_path = "C:/Users/marti/Pictures/lichess_db_standard_rated_2024-09.pgn"
limit_per_class = 50000

# Procesar partidas
df_limited = parse_pgn_and_limit_after(file_path, limit_per_class)

# Verificar tipos de datos y las primeras filas
print("\nTipos de datos en el DataFrame:")
print(df_limited.dtypes)
print("\nPrimeras filas del DataFrame:")
print(df_limited.head())

Leídas 1000 partidas...
Leídas 2000 partidas...
Leídas 3000 partidas...
Leídas 4000 partidas...
Leídas 5000 partidas...
Leídas 6000 partidas...
Leídas 7000 partidas...
Leídas 8000 partidas...
Leídas 9000 partidas...
Leídas 10000 partidas...
Leídas 11000 partidas...
Leídas 12000 partidas...
Leídas 13000 partidas...
Leídas 14000 partidas...
Leídas 15000 partidas...
Leídas 16000 partidas...
Leídas 17000 partidas...
Leídas 18000 partidas...
Leídas 19000 partidas...
Leídas 20000 partidas...
Leídas 21000 partidas...
Leídas 22000 partidas...
Leídas 23000 partidas...
Leídas 24000 partidas...
Leídas 25000 partidas...
Leídas 26000 partidas...
Leídas 27000 partidas...
Leídas 28000 partidas...
Leídas 29000 partidas...
Leídas 30000 partidas...
Leídas 31000 partidas...
Leídas 32000 partidas...
Leídas 33000 partidas...
Leídas 34000 partidas...
Leídas 35000 partidas...
Leídas 36000 partidas...
Leídas 37000 partidas...
Leídas 38000 partidas...
Leídas 39000 partidas...
Leídas 40000 partidas...
Leídas 41

In [ ]:
# Asegurar que la columna elo_class está presente y es de tipo string
if "elo_class" not in df_limited.columns:
    df_limited["elo_class"] = df_limited["elo"].apply(elo_to_class).astype(str)
else:
    df_limited["elo_class"] = df_limited["elo_class"].astype(str)

# Mostrar distribución de clases
df_limited["elo_class"].value_counts().sort_index()

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier

# Asumiendo que 'df_limited' ya está cargado

# Mostrar el número de NaN por columna
print("Número de NaN por columna en df_limited:")
print(df_limited.isnull().sum())

# Eliminar filas con valores NaN
df = df_limited.dropna()
print(f"\nNúmero de filas después de eliminar NaN: {len(df)}")



In [ ]:
# Definir explícitamente las columnas categóricas
cat_features = ['time_control', 'opening_eco', 'opening_name', 'player_color', 'result']
for col in cat_features:
    if col in df.columns:
        df[col] = df[col].astype(str)

# Dividir características y target (para CatBoost)
X = df.drop(['elo', 'elo_class'], axis=1, errors='ignore')
y = df['elo_class']

# Dividir en entrenamiento y prueba (para CatBoost)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\n--- Entrenando y evaluando CatBoost ---")
model_catboost = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function='MultiClass',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50
)

model_catboost.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test)
)

y_pred_catboost = model_catboost.predict(X_test)
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f"\nCatBoost Accuracy: {accuracy_catboost:.4f}")
print("\nReporte de clasificación (CatBoost):\n", classification_report(y_test, y_pred_catboost))

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_test, y_pred_catboost, labels=model_catboost.classes_), annot=True, fmt='d', cmap='Blues',
            xticklabels=model_catboost.classes_, yticklabels=model_catboost.classes_)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - CatBoost')
plt.show()

feature_importances_catboost = model_catboost.get_feature_importance()
importance_df_catboost = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances_catboost})
print("\nImportancia de características (CatBoost):\n", importance_df_catboost.sort_values('Importance', ascending=False).head(10))



In [ ]:
print("\n--- Entrenando y evaluando XGBoost ---")
X_xgb = df.drop(['elo', 'elo_class'], axis=1, errors='ignore')
y_xgb = df['elo_class']
X_xgb = pd.get_dummies(X_xgb, columns=cat_features)
le = LabelEncoder()
y_xgb_encoded = le.fit_transform(y_xgb)
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb, y_xgb_encoded, test_size=0.2, random_state=42, stratify=y_xgb_encoded
)

model_xgb = XGBClassifier(objective='multi:softmax', num_class=len(le.classes_), random_state=42, verbosity=0)
model_xgb.fit(X_train_xgb, y_train_xgb)
y_pred_xgb = model_xgb.predict(X_test_xgb)
y_pred_labels = le.inverse_transform(y_pred_xgb)
y_test_labels = le.inverse_transform(y_test_xgb)

accuracy_xgb = accuracy_score(y_test_labels, y_pred_labels)
print(f"\nXGBoost Accuracy: {accuracy_xgb:.4f}")
print("\nReporte de clasificación (XGBoost):\n", classification_report(y_test_labels, y_pred_labels))

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test_labels, y_pred_labels, labels=le.classes_), annot=True, fmt='d', cmap='Oranges',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Matriz de Confusión - XGBoost')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.tight_layout()
plt.show()

feature_importances_xgb = model_xgb.feature_importances_
importance_df_xgb = pd.DataFrame({'Feature': X_xgb.columns, 'Importance': feature_importances_xgb})
print("\nImportancia de características (XGBoost):\n", importance_df_xgb.sort_values('Importance', ascending=False).head(10))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers

# Asumiendo que 'df_limited' ya está cargado y preprocesado

# --- Random Forest ---
print("\n--- Entrenando y evaluando Random Forest ---")
X_rf = pd.get_dummies(df_limited.drop(['elo', 'elo_class'], axis=1, errors='ignore'), columns=cat_features)
y_rf = df_limited['elo_class']
le_rf = LabelEncoder()
y_rf_encoded = le_rf.fit_transform(y_rf)
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X_rf, y_rf_encoded, test_size=0.2, random_state=42, stratify=y_rf_encoded
)

model_rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1, max_depth=10)
model_rf.fit(X_train_rf, y_train_rf)
y_pred_rf = model_rf.predict(X_test_rf)
y_pred_labels_rf = le_rf.inverse_transform(y_pred_rf)
y_test_labels_rf = le_rf.inverse_transform(y_test_rf)

accuracy_rf = accuracy_score(y_test_labels_rf, y_pred_labels_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print("\nReporte de clasificación (Random Forest):\n", classification_report(y_test_labels_rf, y_pred_labels_rf))

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test_labels_rf, y_pred_labels_rf, labels=le_rf.classes_), annot=True, fmt='d', cmap='Greens',
            xticklabels=le_rf.classes_, yticklabels=le_rf.classes_)
plt.title('Matriz de Confusión - Random Forest')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.tight_layout()
plt.show()

importance_rf = model_rf.feature_importances_
importance_df_rf = pd.DataFrame({'Feature': X_train_rf.columns, 'Importance': importance_rf})
print("\nImportancia de características (Random Forest):\n", importance_df_rf.sort_values('Importance', ascending=False).head(10))

# --- Red Neuronal ---
print("\n--- Entrenando y evaluando Red Neuronal ---")
X_nn = pd.get_dummies(df_limited.drop(['elo', 'elo_class'], axis=1, errors='ignore'), columns=cat_features)
y_nn = df_limited['elo_class']
le_nn = LabelEncoder()
y_nn_encoded = le_nn.fit_transform(y_nn)
num_classes = len(le_nn.classes_)

X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(
    X_nn, y_nn_encoded, test_size=0.2, random_state=42, stratify=y_nn_encoded
)

# Escalar características numéricas para la red neuronal
scaler = StandardScaler()
X_train_scaled_nn = scaler.fit_transform(X_train_nn)
X_test_scaled_nn = scaler.transform(X_test_nn)

# Convertir etiquetas a codificación one-hot
y_train_one_hot_nn = keras.utils.to_categorical(y_train_nn, num_classes=num_classes)
y_test_one_hot_nn = keras.utils.to_categorical(y_test_nn, num_classes=num_classes)

model_nn = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled_nn.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_nn = model_nn.fit(X_train_scaled_nn, y_train_one_hot_nn, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

loss_nn, accuracy_nn = model_nn.evaluate(X_test_scaled_nn, y_test_one_hot_nn, verbose=0)
print(f"Red Neuronal Accuracy: {accuracy_nn:.4f}")

y_pred_proba_nn = model_nn.predict(X_test_scaled_nn)
y_pred_nn = np.argmax(y_pred_proba_nn, axis=1)
y_pred_labels_nn = le_nn.inverse_transform(y_pred_nn)
y_test_labels_nn = le_nn.inverse_transform(np.argmax(y_test_one_hot_nn, axis=1))

print("\nReporte de clasificación (Red Neuronal):\n", classification_report(y_test_labels_nn, y_pred_labels_nn))

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test_labels_nn, y_pred_labels_nn, labels=le_nn.classes_), annot=True, fmt='d', cmap='Purples',
            xticklabels=le_nn.classes_, yticklabels=le_nn.classes_)
plt.title('Matriz de Confusión - Red Neuronal')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Asumiendo que 'df' ya está cargado y preprocesado

# Seleccionar las características específicas
features_puntuacion = [
    'brilliant_moves_ann',
    'blunder_moves_ann',
    'interesting_moves_ann',
    'mistake_moves_ann',
    'inaccuracy_moves_ann',
    'excellent_moves_ann',
    'estimated_accuracy'
]

# Asegurarse de que estas columnas existan en el DataFrame df_limited
for feature in features_puntuacion:
    if feature not in df_limited.columns:
        raise ValueError(f"La columna '{feature}' no se encuentra en el DataFrame df_limited.")

X_puntuacion = df_limited[features_puntuacion]
y_puntuacion = df_limited['elo_class']

# Dividir los datos
X_train_puntuacion, X_test_puntuacion, y_train_puntuacion, y_test_puntuacion = train_test_split(
    X_puntuacion, y_puntuacion, test_size=0.2, random_state=42, stratify=y_puntuacion
)

# Entrenar el modelo CatBoost
model_puntuacion = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function='MultiClass',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50
)

model_puntuacion.fit(
    X_train_puntuacion, y_train_puntuacion,
    eval_set=(X_test_puntuacion, y_test_puntuacion),
    verbose=100
)

# Evaluación del modelo
y_pred_puntuacion = model_puntuacion.predict(X_test_puntuacion)
accuracy_puntuacion = accuracy_score(y_test_puntuacion, y_pred_puntuacion)
print(f"\n--- Modelo con variables de puntuación y precisión ---")
print(f"Accuracy: {accuracy_puntuacion:.4f}")
print("\nReporte de clasificación:\n", classification_report(y_test_puntuacion, y_pred_puntuacion))

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_test_puntuacion, y_pred_puntuacion, labels=model_puntuacion.classes_), annot=True, fmt='d', cmap='viridis',
            xticklabels=model_puntuacion.classes_, yticklabels=model_puntuacion.classes_)
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Modelo con Puntuación y Precisión')
plt.show()

importance_puntuacion = model_puntuacion.get_feature_importance()
importance_df_puntuacion = pd.DataFrame({'Feature': X_puntuacion.columns, 'Importance': importance_puntuacion})
print("\nImportancia de características:\n", importance_df_puntuacion.sort_values('Importance', ascending=False))